In [212]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stat

In [213]:
df = pd.read_csv('bank.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


### 1.모든 column을 독립변수로 활용하여 학습

In [215]:
categorical = [var for var in df.columns if df[var].dtype =='object']
print('categorical:',categorical)

numeric = [var for var in df.columns if df[var].dtype == 'int64']
print('numeric:',numeric)

categorical: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']
numeric: ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [216]:
for var in categorical:
     print(df[var].value_counts()/np.float(len(df)))

management       0.229887
blue-collar      0.174162
technician       0.163322
admin.           0.119513
services         0.082691
retired          0.069701
self-employed    0.036284
student          0.032252
unemployed       0.031984
entrepreneur     0.029385
housemaid        0.024548
unknown          0.006271
Name: job, dtype: float64
married     0.568984
single      0.315176
divorced    0.115839
Name: marital, dtype: float64
secondary    0.490593
tertiary     0.330496
primary      0.134385
unknown      0.044526
Name: education, dtype: float64
no     0.984949
yes    0.015051
Name: default, dtype: float64
no     0.526877
yes    0.473123
Name: housing, dtype: float64
no     0.869199
yes    0.130801
Name: loan, dtype: float64
cellular     0.720480
unknown      0.210177
telephone    0.069342
Name: contact, dtype: float64
may    0.253001
aug    0.136087
jul    0.135639
jun    0.109479
nov    0.084483
apr    0.082691
feb    0.069522
oct    0.035119
jan    0.030819
sep    0.028579
mar    0.0

<ipython-input-216-31f3815c72d6>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(df[var].value_counts()/np.float(len(df)))


In [217]:
for var in categorical:
    print(var, 'contains', df[var].unique())

job contains ['admin.' 'technician' 'services' 'management' 'retired' 'blue-collar'
 'unemployed' 'entrepreneur' 'housemaid' 'unknown' 'self-employed'
 'student']
marital contains ['married' 'single' 'divorced']
education contains ['secondary' 'tertiary' 'primary' 'unknown']
default contains ['no' 'yes']
housing contains ['yes' 'no']
loan contains ['no' 'yes']
contact contains ['unknown' 'cellular' 'telephone']
month contains ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome contains ['unknown' 'other' 'failure' 'success']
deposit contains ['yes' 'no']


In [218]:
df['education'].replace('unknown', np.NAN, inplace=True)
df['contact'].replace('unknown', np.NAN, inplace=True)
df['poutcome'].replace('unknown', np.nan, inplace=True)

In [219]:
df.isnull().sum()

age             0
job             0
marital         0
education     497
default         0
balance         0
housing         0
loan            0
contact      2346
day             0
month           0
duration        0
campaign        0
pdays           0
previous        0
poutcome     8326
deposit         0
dtype: int64

In [220]:
for var in categorical:
    df[var].fillna(df[var].mode()[0], inplace=True)

df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,cellular,5,may,1042,1,-1,0,failure,yes
1,56,admin.,married,secondary,no,45,no,no,cellular,5,may,1467,1,-1,0,failure,yes
2,41,technician,married,secondary,no,1270,yes,no,cellular,5,may,1389,1,-1,0,failure,yes
3,55,services,married,secondary,no,2476,yes,no,cellular,5,may,579,1,-1,0,failure,yes
4,54,admin.,married,tertiary,no,184,no,no,cellular,5,may,673,2,-1,0,failure,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,failure,no
11158,39,services,married,secondary,no,733,no,no,cellular,16,jun,83,4,-1,0,failure,no
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,failure,no
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no


In [221]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [229]:
categorical.remove('deposit')

ValueError: list.remove(x): x not in list

In [255]:
X = df.drop(['deposit'], axis=1)
y = df['deposit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

len(X_train)

8929

In [245]:
# 정확도를 높이기 위해 범주형 변수들을 원-핫 인코딩처리

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

categorical_column = categorical
encoder = OneHotEncoder(drop='first', sparse=False)

X_train_encoded = encoder.fit_transform(X_train[categorical_column])
X_test_encoded = encoder.fit_transform(X_test[categorical_column])

X_train_encoded_rf = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names(categorical_column))
X_test_encoded_rf = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names(categorical_column))

In [246]:
X_train_encoded_rf

,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,...,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8924,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8925,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8926,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [253]:
X_train_new = pd.concat([X_train[numeric], X_train_encoded_rf], axis=1)

X_test_new = pd.concat([X_test[numeric], X_test_encoded_rf], axis=1)

len(X_train_new)

10726

In [248]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train_new[numeric] = scaler.fit_transform(X_train[numeric])

ValueError: Length of values (8929) does not match length of index (10726)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print('Test accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

ValueError: could not convert string to float: 'retired'

In [ ]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train,y_train)
model_pred = clf.predict(X_val)
model_pred


print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_val, model_pred)))

Model accuracy score with 10 decision-trees : 0.8484
